# Load packages

In [ ]:
using Pkg
Pkg.activate("..")

In [ ]:
using Revise
using PHD
using Random, Statistics, CSV, DataFrames, LinearAlgebra

# Setup

- Experimental parameters

In [ ]:
dataset_list = PHD.list_datasets(p_min = 1)
@show length(dataset_list);

In [ ]:
d_num = 6 #51
dname = dataset_list[d_num]
k_missingsignal = 4
k = 10
@show dname, k_missingsignal
SNR = 2

- Read in data

In [ ]:
# Read in a data file.
X_missing = PHD.standardize_colnames(DataFrame(CSV.read("../datasets/"*dname*"/X_missing.csv",
                                                        missingstrings=["", "NaN"])));
@show nrow(X_missing), ncol(X_missing)

In [ ]:
# Remove intrinsic indicators
keep_cols = names(X_missing)
for l in values(PHD.intrinsic_indicators(X_missing, correlation_threshold=1.0))
    # threshold is 1 if optimizing missingness
    setdiff!(keep_cols, l)
end
select!(X_missing, keep_cols)
# indicator of missing features
canbemissing = [any(ismissing.(X_missing[:,j])) for j in names(X_missing)]
# ground truth df
X_full = PHD.standardize_colnames(DataFrame(CSV.read("../datasets/"*dname*"/X_full.csv")))[:,keep_cols];

In [ ]:
X_missing = PHD.optimize_missingness(X_missing, X_full);

In [ ]:
patterns, counts = PHD.missing_patterns_countmap(X_missing)

- Create target

In [ ]:
# Create output
Random.seed!(5234)
@time Y, k, k_missing = PHD.linear_y(X_full, X_missing, k=k, SNR=SNR, canbemissing=canbemissing,
                                     k_missing_in_signal=k_missingsignal, mar=true);
@show k_missing

# Run stuff

In [ ]:
results_main = DataFrame(dataset=[], SNR=[], k=[], kMissing=[], splitnum=[], method=[], osr2=[])
test_prop = .3
Y_predictions = []
test_ind = rand(nrow(X_missing)) .< test_prop ;

for iter in 1:1
    results_table = similar(results_main,0)

    filename = string(dname, "_SNR_", SNR, "_nmiss_", k_missingsignal, "_$iter.csv")

    # Split train / test
    Random.seed!(56802+767*iter)
    test_ind = rand(nrow(X_missing)) .< test_prop ;

    ## Method Oracle
    println("Oracle")
    df = X_full[:,:]
    df[!,:Test] = test_ind
    linear, bestparams = PHD.regress_cv(Y, df, lasso=[true], alpha=collect(0.1:0.1:1))
    R2, OSR2 = PHD.evaluate(Y, df, linear)
    push!(Y_predictions, PHD.predict(df, linear))
    push!(results_table, [dname, SNR, k, k_missing, iter, "Oracle", OSR2])
    
    df = [X_full[:,:] PHD.indicatemissing(X_missing[:,:]; removezerocols=true)]
    df[!,:Test] = test_ind
    linear, bestparams = PHD.regress_cv(Y, df, lasso=[true], alpha=collect(0.1:0.1:1))
    R2, OSR2 = PHD.evaluate(Y, df, linear)
    push!(Y_predictions, PHD.predict(df, linear))
    push!(results_table, [dname, SNR, k, k_missing, iter, "Oracle XM", OSR2])

#     # Method 0
#     println("Method 0")
#     try
#         df = X_missing[:,.!canbemissing] #This step can raise an error if all features can be missing
#         df[!,:Test] = test_ind
#         linear, bestparams = PHD.regress_cv(Y, df, lasso=[true], alpha=collect(0.1:0.1:1))
#         R2, OSR2 = PHD.evaluate(Y, df, linear)
#         push!(Y_predictions, PHD.predict(df, linear))
#         push!(results_table, [dname, SNR, k, k_missing, iter, "Complete Features", OSR2])
#     catch #In this case, simply predict the mean - which leads to 0. OSR2
#         push!(results_table, [dname, SNR, k, k_missing, iter, "Complete Features", 0.])
#     end

#     ## Method 1.1
#     println("Method 1.1")
#     X_imputed = PHD.mice_bruteforce(X_missing);
#     df = deepcopy(X_imputed)
#     df[!,:Test] = test_ind
#     linear, bestparams = PHD.regress_cv(Y, df, lasso=[true], alpha=collect(0.1:0.1:1))
#     push!(Y_predictions, PHD.predict(df, linear))
#     R2, OSR2 = PHD.evaluate(Y, df, linear)
#     push!(results_table, [dname, SNR, k, k_missing, iter, "Imp-then-Reg 1", OSR2])

#     ## Method 1.2
#     println("Method 1.2")
#     df = deepcopy(X_missing)
#     X_train_imputed = PHD.mice_bruteforce(df[.!test_ind,:]);
#     select!(df, names(X_train_imputed))
#     df[.!test_ind,:] .= X_train_imputed
#     X_all_imputed = PHD.mice(df);
#     df = deepcopy(X_all_imputed)
#     df[!,:Test] = test_ind
#     linear, bestparams = PHD.regress_cv(Y, df, lasso=[true], alpha=collect(0.1:0.1:1))
#     push!(Y_predictions, PHD.predict(df, linear))
#     R2, OSR2 = PHD.evaluate(Y, df, linear)
#     push!(results_table, [dname, SNR, k, k_missing, iter, "Imp-then-Reg 2", OSR2])

#     ## Method 1.3
#     println("Method 1.3")
#     df = deepcopy(X_missing)
#     X_train_imputed = PHD.mice_bruteforce(df[.!test_ind,:]);
#     X_all_imputed = PHD.mice_bruteforce(df[:,names(X_train_imputed)]);
#     select!(df, names(X_train_imputed))
#     df[.!test_ind,:] .= X_train_imputed
#     df[test_ind,:] .= X_all_imputed[test_ind,:]
#     df[!,:Test] = test_ind
#     linear, bestparams = PHD.regress_cv(Y, df, lasso=[true], alpha=collect(0.1:0.1:1))
#     push!(Y_predictions, PHD.predict(df, linear))
#     R2, OSR2 = PHD.evaluate(Y, df, linear)
#     push!(results_table, [dname, SNR, k, k_missing, iter, "Imp-then-Reg 3", OSR2])

    ## Method 1.4
    println("Method 1.4")
    means_df = PHD.compute_mean(X_missing[.!test_ind,:])
    X_imputed = PHD.mean_impute(X_missing, means_df);
    df = deepcopy(X_imputed)
    df[!,:Test] = test_ind
    linear, bestparams = PHD.regress_cv(Y, df, lasso=[true], alpha=collect(0.1:0.1:1))
    push!(Y_predictions, PHD.predict(df, linear))
    R2, OSR2 = PHD.evaluate(Y, df, linear)
    push!(results_table, [dname, SNR, k, k_missing, iter, "Imp-then-Reg 4", OSR2])

#     ## Method 1.5 Mean and mode impute
#     println("Method 1.5")
#     means_df = PHD.compute_mean(X_missing[.!test_ind,:])
#     X_imputed = PHD.mean_impute(X_missing, means_df);
#     df = deepcopy(X_imputed)
#     PHD.mode_impute!(df, train = .!test_ind)
#     df[!,:Test] = test_ind
#     linear, bestparams = PHD.regress_cv(Y, df, lasso=[true], alpha=collect(0.1:0.1:1))
#     push!(Y_predictions, PHD.predict(df, linear))
#     R2, OSR2 = PHD.evaluate(Y, df, linear)
#     push!(results_table, [dname, SNR, k, k_missing, iter, "Imp-then-Reg 5", OSR2])

    ## Method 2: Static Adaptability
    println("Method 2")
    df = deepcopy(X_missing)
    df[!,:Test] = test_ind
    X_augmented = hcat(PHD.zeroimpute(df), PHD.indicatemissing(df, removezerocols=true))
    @time linear2, bestparams2 = PHD.regress_cv(Y, X_augmented, lasso=[true],
                                            alpha=collect(0.1:0.1:1),
                                            missing_penalty=[2.0,4.0,6.0,8.0,12.0,16.0])
    push!(Y_predictions, PHD.predict(X_augmented, linear2))
    R2, OSR2 = PHD.evaluate(Y, X_augmented, linear2)
    push!(results_table, [dname, SNR, k, k_missing, iter, "Static", OSR2])

    ## Method 3: Affine Adaptability
    println("Method 3")
    df = deepcopy(X_missing)
    df[!,:Test] = test_ind
    X_affine = PHD.augmentaffine(df, removezerocols=true)
    @time linear3, bestparams3 = PHD.regress_cv(Y, X_affine, lasso=[true], alpha=collect(0.1:0.1:1),
                                          missing_penalty=[2.0,4.0,6.0,8.0,12.0,16.0])
    push!(Y_predictions, PHD.predict(X_affine, linear3))
    R2, OSR2 = PHD.evaluate(Y, X_affine, linear3)
    push!(results_table, [dname, SNR, k, k_missing, iter, "Affine", OSR2])

    results_main = vcat(results_main, results_table)
end

In [ ]:
results_main

In [ ]:
missing_rows = vec(any(ismissing.(X_missing) |> Matrix, dims=2))
for i = 1:5
    @show i
    @show sum((Y[missing_rows] .- Y_predictions[i][missing_rows]) .^ 2)
    @show sum((Y[.!missing_rows] .- Y_predictions[i][.!missing_rows]) .^ 2)
end
@show sum(missing_rows), sum(.!missing_rows)